In [26]:
#Import the necessary libraries
import os
import py2exe
import pickle
import pathlib
import webbrowser
import matplotlib
import numpy as np
import pandas as pd
import tkinter as tk
import seaborn as sns
import pyswarms as ps
from PIL import Image
import tensorflow as tf
from sklearn import svm
from scipy import interp
from pathlib import Path
from itertools import cycle
from sklearn.svm import SVC
from sklearn import metrics
from numpy import genfromtxt
from subprocess import Popen
import statsmodels.api as sm
import tkinter.font as tkFont
from scipy.linalg import lstsq
import matplotlib.pyplot as plt
from matplotlib import rcParams
from distutils.core import setup
import tensorflow_probability as tfp
from pykrige.ok import OrdinaryKriging
from tensorflow.keras.layers import Input
from sklearn.metrics import roc_curve, auc
from matplotlib.colors import ListedColormap
from sklearn.metrics import confusion_matrix
from gplearn.genetic import SymbolicClassifier
from tkinter import ttk, filedialog, messagebox
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import Figure, MultipleLocator
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tensorflow.keras.models import Model, Sequential, load_model
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

config = {"font.family":'Times New Roman',  "font.size": 10,}
rcParams.update(config)
tfd = tfp.distributions

In [27]:
#Define the main menu
class Main_menu(tk.Tk):
    def __init__(self):
        super().__init__() 
        self.createWidgets()
        
    def createWidgets(self):
        self.title('Open stope stability prediction')
        self.columnconfigure(0, minsize=200)
        self.geometry('750x280')
        topframe = tk.Frame(self, height=20)
        topframe.pack(fill="both", expand="yes", side=tk.TOP)
        path = pathlib.Path('__file__').parent.absolute()
        path_main_interface = os.path.join(path, 'Interface.png')
        self.Main_interface_image = tk.PhotoImage(file=path_main_interface)
        Main_interface = tk.Label(topframe, image=self.Main_interface_image)
        Main_interface.place(x=0, y=0, relwidth=1, relheight=1)
        self.menubar = tk.Menu()
        configuration = tk.Menu(self.menubar, tearoff=0)
        configuration.add_command(label="Stability prediction based on machine learning", command=self.config1)
        configuration.add_command(label="Stability analysis criterion", command=self.config2)
        self.menubar.add_cascade(label="Function selection", menu=configuration)
        self.menubar.add_cascade(label="Quit", command=quit)
        self.config(menu=self.menubar)   
        
    def config1(self):
        self.destroy()
        ML_technology()
        
    def config2(self):
        self.destroy()
        GP_technology()  

In [28]:
#Open stope stability prediction based on machine learning model
class ML_technology(tk.Tk):
    def __init__(self):
        super().__init__()
        self.createWidgets()
        
    def createWidgets(self):
        self.title('Open stope stability prediction based on machine learning')
        self.columnconfigure(0, minsize=100)
        self.geometry('750x750') 
        global clicked
        clicked = False
        fir_font = tkFont.Font(family="Times new roman", size=15, weight="normal")
        sec_font = tkFont.Font(family="Times new roman", size=14, weight="normal")
        thi_font = tkFont.Font(family="Times new roman", size=45, weight="bold")
        fou_font = tkFont.Font(family="Times new roman", size=17, weight="normal")
        topframe = tk.Frame(self, height=80)
        topframe.pack(fill="both", expand="yes", side=tk.TOP)
        inputframe = tk.Frame(self, height=80)
        inputframe.pack(fill="both", expand="yes", side=tk.TOP)
        predictframe = tk.Frame(self, height=80)
        predictframe.pack(fill="both", expand="yes", side=tk.TOP)
        Visualizationframe = tk.Frame(self, height=80)
        Visualizationframe.pack(fill="both", expand="yes", side=tk.TOP)
        labelframe1 = tk.LabelFrame(topframe, font=fir_font,background="#FFA07A")
        labelframe1.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe3 = tk.LabelFrame(inputframe, text="Initialization", font=fir_font,background="#ADD8E6")
        labelframe3.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe4 = tk.LabelFrame(inputframe, text="Confusion matrix", font=fir_font,background="#ADD8E6")
        labelframe4.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe5 = tk.LabelFrame(inputframe, text="Test performance", font=fir_font,background="#ADD8E6")
        labelframe5.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe6 = tk.LabelFrame(predictframe, text="Input", font=fir_font,background="#FFFACD")
        labelframe6.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe7 = tk.LabelFrame(predictframe, text="Stability grade", font=fir_font,background="#FFFACD")
        labelframe7.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe8 = tk.LabelFrame(Visualizationframe, text="Performance visualization (ROC)", font=fir_font)
        labelframe8.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe9 = tk.LabelFrame(Visualizationframe, text="Update the stability graph", font=fir_font)
        labelframe9.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        self.entryvar = tk.StringVar()
        tk.Label(labelframe1, text='Path：', font=fir_font).grid(row=0, column=0, sticky=tk.W)
        tk.Entry(labelframe1, width=32,bd=4,textvariable=self.entryvar).grid(row=0, column=1)
        tk.Button(labelframe1, command=self.__openfile, text='Dataset loading', font=fir_font). \
            grid(row=0, column=2, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Training, text='Modeling', font=fir_font) \
            .grid(row=0, column=3, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Predict, text='Predict', font=fir_font) \
            .grid(row=0, column=4, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Main_interface, text='Quit', font=fir_font) \
            .grid(row=0, column=5, padx=10, pady=10)
        self.SVM_Test_size_Parameter = tk.StringVar()
        self.SVM_C_Parameter = tk.StringVar()
        self.SVM_gamma_Parameter = tk.StringVar()
        self.SVM_Random_State_Parameter = tk.StringVar()
        ent_Test_size=tk.Entry(labelframe3, width=14, textvariable=self.SVM_Test_size_Parameter, validate='key', fg='gray', font=sec_font)
        ent_Test_size.grid(row=1, column=2, padx=4, pady=11)
        ent_Test_size.insert(0, '0.25')
        self.Default_hint(ent_Test_size)
        ent_C=tk.Entry(labelframe3, width=14, textvariable=self.SVM_C_Parameter, validate='key', fg='gray', font=sec_font)
        ent_C.grid(row=2, column=2, padx=4, pady=11)
        ent_C.insert(0, '5.26546597e+00')
        self.Default_hint(ent_C)
        ent_gamma=tk.Entry(labelframe3, width=14, textvariable=self.SVM_gamma_Parameter, validate='key', fg='gray', font=sec_font)
        ent_gamma.grid(row=3, column=2, padx=4, pady=11)
        ent_gamma.insert(0, '2.31533143e-03')
        self.Default_hint(ent_gamma)
        ent_SVM_Random_State=tk.Entry(labelframe3, width=14, textvariable=self.SVM_Random_State_Parameter, validate='key', fg='gray', font=sec_font)
        ent_SVM_Random_State.grid(row=4, column=2, padx=4, pady=11)
        ent_SVM_Random_State.insert(0, '0')
        self.Default_hint(ent_SVM_Random_State)
        lab_Test_size=tk.Label(labelframe3, text='Dataset split ratio', font=sec_font)
        lab_Test_size.grid(row=1, column=1, sticky='w', padx=4, pady=3)
        self.CreateToolTip(lab_Test_size,text='The proportion of test sets in the input data set')
        lab_C=tk.Label(labelframe3, text='Parameter C', font=sec_font)
        lab_C.grid(row=2, column=1, sticky='w', padx=4, pady=3)
        self.CreateToolTip(lab_C, text='Control the degree of punishment for misclassified samples')
        lab_gamma=tk.Label(labelframe3, text='Parameter gamma', font=sec_font)
        lab_gamma.grid(row=3, column=1, sticky='w', padx=4, pady=3)
        self.CreateToolTip(lab_gamma, text='Affects the complexity of the model')
        lab_SVM_Random_State=tk.Label(labelframe3, text='Random number', font=sec_font)
        lab_SVM_Random_State.grid(row=4, column=1, sticky='w', padx=4, pady=3)
        self.CreateToolTip(lab_SVM_Random_State, text='The number of random seeds in the SVM model')
        self.ACC_train = tk.StringVar()
        self.PRE_train = tk.StringVar()
        self.REC_train = tk.StringVar()
        self.F1_train = tk.StringVar()
        self.ACC_test = tk.StringVar()
        self.PRE_test = tk.StringVar()
        self.REC_test = tk.StringVar()
        self.F1_test = tk.StringVar() 
        self.SVM_fig3 = Figure(figsize=(2, 2), dpi=100,frameon=True,edgecolor = "#FFF5EE")  # weight*height
        self.SVM_canvas3 = FigureCanvasTkAgg(self.SVM_fig3, master=labelframe4)
        self.SVM_canvas3.get_tk_widget().grid(column = 1, row = 1)
        self.SVM_canvas3._tkcanvas.pack(side=tk.TOP, fill=tk.X, expand=1)
        tk.Entry(labelframe5, width=12, textvariable=self.ACC_test, validate='key', state=tk.DISABLED, font=sec_font).grid(row=1,column=5, padx=5, pady=3)
        tk.Entry(labelframe5, width=12, textvariable=self.PRE_test, validate='key', state=tk.DISABLED, font=sec_font).grid(row=2,column=5, padx=5, pady=3)
        tk.Entry(labelframe5, width=12, textvariable=self.REC_test, validate='key', state=tk.DISABLED, font=sec_font).grid(row=3, column=5, padx=5, pady=3)
        tk.Entry(labelframe5, width=12, textvariable=self.F1_test, validate='key', state=tk.DISABLED, font=sec_font).grid(row=4, column=5, padx=5, pady=3)
        tk.Label(labelframe5, text='Accuracy', font=sec_font).grid(row=1, column=6, sticky='e', padx=5, pady=11)
        tk.Label(labelframe5, text='Precision', font=sec_font).grid(row=2, column=6, sticky='e', padx=5, pady=11)
        tk.Label(labelframe5, text='Recall', font=sec_font).grid(row=3, column=6, sticky='e', padx=5, pady=11)
        tk.Label(labelframe5, text='F1-Score', font=sec_font).grid(row=4, column=6, sticky='e', padx=5, pady=11)
        self.HR_Input = tk.StringVar()
        self.N_Input = tk.StringVar()
        self.SVM_Stability_Degree = tk.StringVar()
        ent1=tk.Entry(labelframe6, textvariable=self.HR_Input, validate='key', bg='#FFDEAD',width=6,bd=4, font=fou_font,justify='center')
        ent1.grid(row=1, column=2, padx=5, pady=5)
        self.Default_hint(ent1)
        ent2=tk.Entry(labelframe6,  textvariable=self.N_Input, validate='key', bg='#FFDEAD',width=6,bd=4,font=fou_font,justify='center')
        ent2.grid(row=3, column=2, padx=5, pady=5)
        self.Default_hint(ent2)
        lab1=tk.Label(labelframe6, text='Hydraulic radius', font=fou_font,bg='#FFDEAD')
        lab1.grid(row=1, column=1, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab1,text='Ratio of surface area to surface circumference of exposed surface of stope')
        lab2=tk.Label(labelframe6, text='Stability number', font=fou_font,bg='#FFDEAD')
        lab2.grid(row=3, column=1, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab2, text='A comprehensive index used to evaluate the stability of rock mass under given stress conditions')
        tk.Entry(labelframe7, width=14, textvariable=self.SVM_Stability_Degree, validate='key', state=tk.DISABLED,font=thi_font,justify='center').grid(row=1, column=4, padx=20, pady=20) 
        self.SVM_fig1 = Figure(figsize=(3.1, 3.3), dpi=100,frameon=True,edgecolor = "#FFF5EE")  # weight*height
        self.SVM_canvas1 = FigureCanvasTkAgg(self.SVM_fig1, master=labelframe8)
        self.SVM_canvas1.get_tk_widget().grid(column = 1, row = 1)
        self.SVM_canvas1._tkcanvas.pack(side=tk.TOP, fill=tk.X, expand=1)
        self.SVM_fig2 = Figure(figsize=(4, 3.3), dpi=100,frameon=True,edgecolor = "#FFF5EE")
        self.SVM_canvas2 = FigureCanvasTkAgg(self.SVM_fig2, master=labelframe9)
        self.SVM_canvas2.get_tk_widget().grid(column = 1, row = 1)
        self.SVM_canvas2._tkcanvas.pack(side=tk.TOP, fill=tk.X, expand=1)
        self.SVM_ax1 = self.SVM_fig1.add_subplot(111)
        self.SVM_fig1.tight_layout(h_pad=2,w_pad=2)
        self.SVM_ax2 = self.SVM_fig2.add_subplot(111)
        self.SVM_fig2.tight_layout(h_pad=2,w_pad=2)
        self.SVM_ax3 = self.SVM_fig3.add_subplot(111)
        self.SVM_fig3.tight_layout(h_pad=1,w_pad=1)

    def Main_interface(self):
        self.destroy()
        Main_menu()

    def Default_hint(self, widget):
        global clicked
        def hint(event):
            if (clicked == False):
                widget.delete(0, tk.END)
                widget.config(fg="black")
        widget.bind("<Button-1>", hint)

    def CreateToolTip(self, widget, text):
        toolTip = ToolTip(widget)
        def enter(event):
            toolTip.showtip(text)
        def leave(event):
            toolTip.hidetip()
        widget.bind('<Enter>', enter)
        widget.bind('<Leave>', leave)

    def __openfile(self):
        self.filename = filedialog.askopenfilename(title='open csv file', filetype=[('csv', '*.csv')])
        self.entryvar.set(self.filename)
        if not self.filename:
            messagebox.showwarning('Warning', message='File not yet selected！')

    def Training(self):
        dataset = pd.read_csv(self.filename)        
        X=dataset.iloc[:,0:2]
        y=dataset.iloc[:,2] 
        self.SVM_x0_0=dataset.iloc[0:312,0:1]
        self.SVM_x1_0=dataset.iloc[0:312,1:2]
        self.SVM_x0_1=dataset.iloc[312:466,0:1]
        self.SVM_x1_1=dataset.iloc[312:466,1:2]
        SVM_Test_size=float(self.SVM_Test_size_Parameter.get())
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=SVM_Test_size,random_state=79)
        self.SVM_n_samples, self.SVM_n_features = X.shape
        # search optimum configuration
        SVM_C=float(self.SVM_C_Parameter.get())
        SVM_gamma=float(self.SVM_gamma_Parameter.get())
        SVM_Random_State=int(self.SVM_Random_State_Parameter.get())
        SVM_model = svm.SVC(kernel='rbf',C=SVM_C,gamma = SVM_gamma,random_state=SVM_Random_State)
        SVM_model.fit(X_train, y_train)
        train_PredictY= SVM_model.predict(X_train)

        trainY_True = y_train
        self.SVM_y_true = y_test
        self.SVM_y_proba = SVM_model.predict(X_test)
        test_PredictY = SVM_model.predict(X_test)
        testY_True = y_test
        self.test_result = np.hstack((testY_True, test_PredictY))
        test_ACC = accuracy_score(testY_True, test_PredictY)
        test_PRE = precision_score(testY_True, test_PredictY,pos_label=0)
        test_REC = recall_score(testY_True, test_PredictY,pos_label=0)
        test_F1 = f1_score(testY_True, test_PredictY,pos_label=0)
        self.ACC_test.set(str(test_ACC))
        self.PRE_test.set(str(test_PRE))
        self.REC_test.set(str(test_REC))
        self.F1_test.set(str(test_F1))
        self.SVM_Visualization_confusion_matrix(self.SVM_y_true, self.SVM_y_proba)
        self.SVM_Visualization_ROC(self.SVM_y_true, self.SVM_y_proba,self.SVM_n_samples,self.SVM_n_features)
        SVM_DecisionBoundary_axis=[1, 50, 0.1, 1000]
        self.SVM_DecisionBoundary_X0, self.SVM_DecisionBoundary_X1 = np.meshgrid(np.linspace(SVM_DecisionBoundary_axis[0], SVM_DecisionBoundary_axis[1], 100).reshape(-1, 1),
                                                                                         np.linspace(SVM_DecisionBoundary_axis[2], SVM_DecisionBoundary_axis[3], 1000).reshape(-1, 1),)
        self.SVM_X_grid_matrix = np.c_[self.SVM_DecisionBoundary_X0.ravel(), self.SVM_DecisionBoundary_X1.ravel()]
        self.SVM_y_predict_DecisionBoundary = SVM_model.predict(self.SVM_X_grid_matrix)
        self.SVM_y_predict_matrix_DecisionBoundary = self.SVM_y_predict_DecisionBoundary.reshape(self.SVM_DecisionBoundary_X0.shape)
        self.SVM_Visualization_DecisionBoundary(self.SVM_x0_0,self.SVM_x1_0,self.SVM_x0_1,self.SVM_x1_1,
                                       self.SVM_y_predict_matrix_DecisionBoundary,self.SVM_DecisionBoundary_X0,self.SVM_DecisionBoundary_X1)
 
    def Predict(self):
        Stability_Graph_HR=float(self.HR_Input.get())
        Stability_Graph_N=float(self.N_Input.get())
        dataset = pd.read_csv(self.filename)        
        X=dataset.iloc[:,0:2]
        y=dataset.iloc[:,2] 
        SVM_Test_size=float(self.SVM_Test_size_Parameter.get())
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=SVM_Test_size,random_state=19)
        SVM_C=float(self.SVM_C_Parameter.get())
        SVM_gamma=float(self.SVM_gamma_Parameter.get())
        SVM_Random_State=int(self.SVM_Random_State_Parameter.get())
        SVM_model = svm.SVC(kernel='rbf',C=SVM_C,gamma = SVM_gamma,random_state=SVM_Random_State)
        SVM_model.fit(X_train, y_train)
        train_PredictY= SVM_model.predict(X_train)
        new_feature=[Stability_Graph_HR,Stability_Graph_N]
        X_new=np.array([new_feature])
        prediction=SVM_model.predict(X_new)
        if prediction == 0:
            SVM_prediction_category="Stable"
        if prediction == 1:
            SVM_prediction_category="Unstable"
        self.SVM_Stability_Degree.set(SVM_prediction_category)
    
    def SVM_Visualization_confusion_matrix(self, SVM_y_true, SVM_y_proba):
        labels = [0, 1]
        confusion = confusion_matrix(SVM_y_true, SVM_y_proba)
        self.SVM_ax3.imshow(confusion, cmap=plt.cm.Blues)
        indices = range(len(confusion))
        for first_index in range(len(confusion)): 
            for second_index in range(len(confusion)):   
                self.SVM_ax3.text(first_index, second_index, confusion[first_index][second_index],color='red', fontsize=20, va='center', ha='center')
        self.SVM_ax3.set_xticks([0,1])   
        self.SVM_ax3.set_yticks([0,1])  
        self.SVM_canvas3.draw()

    def SVM_Visualization_ROC(self, SVM_y_true, SVM_y_proba,SVM_n_samples,SVM_n_features):
        fpr, tpr, threshold = metrics.roc_curve(SVM_y_true, SVM_y_proba)
        roc_auc = metrics.auc(fpr, tpr)
        self.SVM_ax1.plot(fpr, tpr, 'r',label = "AUC = %0.4f" % roc_auc,linewidth=2)
        self.SVM_ax1.plot([0, 1], [0, 1], 'k--', lw=1)
        self.SVM_ax1.set_xlim([-0.05, 1.0])
        self.SVM_ax1.set_ylim([-0.05, 1.05])
        self.SVM_ax1.tick_params(direction='in',which='major',width=1,length=4,pad = 0.9)
        self.SVM_ax1.set_xticks([0,0.2,0.4,0.6,0.8,1])   
        self.SVM_ax1.set_yticks([0,0.2,0.4,0.6,0.8,1])        
        self.SVM_ax1.set_xlabel('FPR', fontsize=12)
        self.SVM_ax1.set_ylabel('TPR', fontsize=12)
        self.SVM_ax1.legend(loc="lower right",fontsize=9,facecolor='cornsilk', edgecolor='blue')
        self.SVM_canvas1.draw()
 
    def SVM_Visualization_DecisionBoundary(self,SVM_x0_0,SVM_x1_0,SVM_x0_1,SVM_x1_1,
                                       SVM_y_predict_matrix_DecisionBoundary,SVM_DecisionBoundary_X0,SVM_DecisionBoundary_X1):           
        my_colormap = ListedColormap(['#a2bffe', '#FFC0CB'])
        self.SVM_ax2.contourf(SVM_DecisionBoundary_X0, SVM_DecisionBoundary_X1, SVM_y_predict_matrix_DecisionBoundary,cmap=my_colormap)

        self.SVM_ax2.tick_params(direction='in',which='major',width=1,length=4,pad = 0.7)
        self.SVM_ax2.tick_params(direction='in',which='minor',width=1,length=2,pad = 0.7)

        self.SVM_ax2.scatter(SVM_x0_0, SVM_x1_0, color="#0000CD",s = 5)
        self.SVM_ax2.scatter(SVM_x0_1, SVM_x1_1, color="#A52A2A",s = 5)
        self.SVM_ax2.set_xscale('log')
        self.SVM_ax2.set_yscale('log')
        self.SVM_ax2.set_xlabel('Hydraulic Radius (m)',size=12 ,labelpad = 0.5)
        self.SVM_ax2.set_ylabel('Stability Number, N',size=10, labelpad = 0.5)
        self.SVM_canvas2.draw()  
        
class ToolTip(object):
    def __init__(self, widget):
        self.widget = widget
        self.tipwindow = None
        self.id = None
        self.x = self.y = 0

    def showtip(self, text):
        self.text = text
        if self.tipwindow or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 20
        y = y + cy + self.widget.winfo_rooty() +20
        self.tipwindow = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(1)
        tw.wm_geometry("+%d+%d" % (x, y))
        label = tk.Label(tw, text=self.text, justify=tk.LEFT,
                      background="#ffffe0", relief=tk.SOLID, borderwidth=1,
                      font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hidetip(self):
        tw = self.tipwindow
        self.tipwindow = None
        if tw:
            tw.destroy()

In [29]:
#Stope stability prediction expression based on genetic programming (GP) technology
class GP_technology(tk.Tk):
    def __init__(self):
        super().__init__()
        self.createWidgets()

    def createWidgets(self):
        self.title('Open stope stability prediction expression based on genetic programming (GP)')
        self.columnconfigure(0, minsize=100)
        self.geometry('750x560') 
        global clicked
        clicked = False
        fir_font = tkFont.Font(family="Times new roman", size=15, weight="normal")
        sec_font = tkFont.Font(family="Times new roman", size=14, weight="normal")
        thi_font = tkFont.Font(family="Times new roman", size=17, weight="bold")
        fou_font = tkFont.Font(family="Times new roman", size=17, weight="normal")
        fiv_font = tkFont.Font(family="Times new roman", size=15, weight="bold")
        Firstframe = tk.Frame(self, height=80)
        Firstframe.pack(fill="both", expand="yes", side=tk.TOP)
        Firstframe2 = tk.Frame(self, height=80)
        Firstframe2.pack(fill="both", expand="yes", side=tk.TOP)
        Secondframe = tk.Frame(self, height=80)
        Secondframe.pack(fill="both", expand="yes", side=tk.TOP)
        Secondframe2 = tk.Frame(self, height=80)
        Secondframe2.pack(fill="both", expand="yes", side=tk.TOP)
        Thirdframe = tk.Frame(self, height=80)
        Thirdframe.pack(fill="both", expand="yes", side=tk.TOP)
        Fouthframe = tk.Frame(self, height=80)
        Fouthframe.pack(fill="both", expand="yes", side=tk.TOP)
        labelframe1 = tk.LabelFrame(Firstframe, font=fir_font,background="#FFA07A")
        labelframe1.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe2 = tk.LabelFrame(Firstframe2, text="Fit the CS-SVM decision boundary", font=fir_font,background="#FFFACD")
        labelframe2.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe3 = tk.LabelFrame(Secondframe, text="Parameter setting of GP technology", font=fir_font,background="#ADD8E6")
        labelframe3.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe4 = tk.LabelFrame(Secondframe, text="Performance", font=fir_font,background="#ADD8E6")
        labelframe4.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe5 = tk.LabelFrame(Secondframe2, text="Derive the GP prediction expression", font=fir_font,background="#FFFACD")
        labelframe5.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe6 = tk.LabelFrame(Thirdframe,text="Stability identification criteria", font=fir_font,background="#FFA07A")
        labelframe6.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        self.GP_entryvar = tk.StringVar()
        tk.Label(labelframe1, text='Dataset path：', font=fir_font).grid(row=0, column=0, sticky=tk.W)
        tk.Entry(labelframe1, width=32,bd=4,textvariable=self.GP_entryvar).grid(row=0, column=1)
        tk.Button(labelframe1, command=self.GP_openfile, text='Dataset loading', font=fir_font). \
            grid(row=0, column=2, padx=13, pady=5)
        tk.Button(labelframe1,text='GP program',command=self.GP_Stope_Prediction, font=fir_font) \
            .grid(row=0, column=3, padx=13, pady=5)
        tk.Button(labelframe1, command=self.Main_interface, text='Quit', font=fir_font) \
            .grid(row=0, column=4, padx=13, pady=5)
        tk.Label(labelframe2,text='Exp. Y1: Y₁(HR, N) = -3.95 + 16.48HR - 22.83HR² + 14.55HR³ - 3.33HR⁴ - N',font=fiv_font).grid(row=0, column=0, sticky=tk.W,padx=45, pady=5)
        self.EST_population_size_Parameter = tk.StringVar()
        self.EST_generations_Parameter = tk.StringVar()
        self.EST_PCrossover_Parameter = tk.StringVar()
        self.EST_PSubtreeMutation_Parameter = tk.StringVar()
        self.EST_PHoistMutation_Parameter = tk.StringVar()
        self.EST_PPointMutation_Parameter = tk.StringVar()    
        ent_EST_population_size=tk.Entry(labelframe3, width=6, textvariable=self.EST_population_size_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_population_size.grid(row=1, column=2, padx=3, pady=4)
        ent_EST_population_size.insert(0, '0')
        self.Default_hint(ent_EST_population_size)
        ent_EST_generations=tk.Entry(labelframe3, width=6, textvariable=self.EST_generations_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_generations.grid(row=2, column=2, padx=3, pady=4)
        ent_EST_generations.insert(0, '0')
        self.Default_hint(ent_EST_generations)
        ent_EST_PCrossover=tk.Entry(labelframe3, width=6, textvariable=self.EST_PCrossover_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_PCrossover.grid(row=3, column=2, padx=3, pady=4)
        ent_EST_PCrossover.insert(0, '0')
        self.Default_hint(ent_EST_PCrossover)
        ent_EST_PSubtreeMutation=tk.Entry(labelframe3, width=6, textvariable=self.EST_PSubtreeMutation_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_PSubtreeMutation.grid(row=1, column=4, padx=3, pady=4)
        ent_EST_PSubtreeMutation.insert(0, '0')
        self.Default_hint(ent_EST_PSubtreeMutation)
        ent_EST_PHoistMutation=tk.Entry(labelframe3, width=6, textvariable=self.EST_PHoistMutation_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_PHoistMutation.grid(row=2, column=4, padx=3, pady=4)
        ent_EST_PHoistMutation.insert(0, '0')
        self.Default_hint(ent_EST_PHoistMutation)
        ent_EST_PPointMutation=tk.Entry(labelframe3, width=6, textvariable=self.EST_PPointMutation_Parameter, validate='key', fg='gray', font=sec_font)
        ent_EST_PPointMutation.grid(row=3, column=4, padx=3, pady=4)
        ent_EST_PPointMutation.insert(0, '0')
        self.Default_hint(ent_EST_PPointMutation)
        lab_EST_population_size=tk.Label(labelframe3, text='Population', font=sec_font)
        lab_EST_population_size.grid(row=1, column=1, sticky='w', padx=4, pady=4)
        lab_EST_generations=tk.Label(labelframe3, text='Iterations', font=sec_font)
        lab_EST_generations.grid(row=2, column=1, sticky='w', padx=4, pady=4)
        lab_EST_PCrossover=tk.Label(labelframe3, text='P_crossover', font=sec_font)
        lab_EST_PCrossover.grid(row=3, column=1, sticky='w', padx=4, pady=4)
        lab_EST_PSubtreeMutation=tk.Label(labelframe3, text='P_subtree_mutation', font=sec_font)
        lab_EST_PSubtreeMutation.grid(row=1, column=3, sticky='w', padx=4, pady=4)
        lab_EST_PHoistMutation=tk.Label(labelframe3, text='P_hoist_mutation', font=sec_font)
        lab_EST_PHoistMutation.grid(row=2, column=3, sticky='w', padx=4, pady=4)
        lab_EST_PPointMutation=tk.Label(labelframe3, text='P_point_mutation', font=sec_font)
        lab_EST_PPointMutation.grid(row=3, column=3, sticky='w', padx=4, pady=4)
        self.ACC_Display_GP = tk.StringVar()
        self.PRE_Display_GP = tk.StringVar()
        self.F1_Display_GP = tk.StringVar()
        self.REC_Display_GP = tk.StringVar()
        self.AUC_Display_GP = tk.StringVar()
        tk.Entry(labelframe4, width=8, textvariable=self.ACC_Display_GP, validate='key', state=tk.DISABLED, font=sec_font).grid(row=1,column=5, padx=4, pady=4)        
        tk.Entry(labelframe4, width=8, textvariable=self.PRE_Display_GP, validate='key', state=tk.DISABLED, font=sec_font).grid(row=2,column=5, padx=4, pady=4)
        tk.Entry(labelframe4, width=8, textvariable=self.F1_Display_GP, validate='key', state=tk.DISABLED, font=sec_font).grid(row=3, column=5, padx=4, pady=4)
        tk.Entry(labelframe4, width=8, textvariable=self.REC_Display_GP, validate='key', state=tk.DISABLED, font=sec_font).grid(row=1, column=7, padx=4, pady=4)
        tk.Entry(labelframe4, width=8, textvariable=self.AUC_Display_GP, validate='key', state=tk.DISABLED, font=sec_font).grid(row=2,column=7, padx=4, pady=4)
        tk.Label(labelframe4, text='Accuracy', font=sec_font).grid(row=1, column=4, sticky='w', padx=5, pady=4)
        tk.Label(labelframe4, text='Precision', font=sec_font).grid(row=2, column=4, sticky='w', padx=5, pady=4)
        tk.Label(labelframe4, text='F1-Score', font=sec_font).grid(row=3, column=4, sticky='w', padx=5, pady=4)
        tk.Label(labelframe4, text='Recall', font=sec_font).grid(row=1, column=6, sticky='w', padx=5, pady=4)
        tk.Label(labelframe4, text='AUC', font=sec_font).grid(row=2, column=6, sticky='w', padx=5, pady=4)                   
        tk.Label(labelframe5,text='Exp. Y2: 1/(1+e^(-F))',font=fiv_font).grid(row=0, column=0, sticky=tk.W,padx=6, pady=4)
        tk.Label(labelframe5,text='Where F(HR, N)=',font=fiv_font).grid(row=0, column=1, sticky=tk.W,padx=6, pady=4)
        self.GP_Expression_Output = tk.StringVar()        
        tk.Entry(labelframe5, width=29, textvariable=self.GP_Expression_Output, validate='key',
                 state=tk.DISABLED,font=thi_font,justify='center').grid(row=0, column=2, padx=6, pady=10)
        Expression_path = pathlib.Path('__file__').parent.absolute()
        path_expression = os.path.join(Expression_path, 'Criterion.png')
        self.Expression_image = tk.PhotoImage(file=path_expression)
        self.Expression_image = self.Expression_image.subsample(1, 1)
        Expression_Y1Y2 = tk.Label(labelframe6, image=self.Expression_image).grid(row=0, column=0, sticky=tk.W,padx=4, pady=5)

    def GP_Stope_Prediction(self):
        self.MSG_GP_Expression()        

    def Main_interface(self):
        self.destroy()
        Main_menu()

    def Default_hint(self, widget):
        global clicked
        def hint(event):
            if (clicked == False):
                widget.delete(0, tk.END)
                widget.config(fg="black")
        widget.bind("<Button-1>", hint)

    def CreateToolTip(self, widget, text):
        toolTip = ToolTip(widget)
        def enter(event):
            toolTip.showtip(text)
        def leave(event):
            toolTip.hidetip()
        widget.bind('<Enter>', enter)
        widget.bind('<Leave>', leave)

    def GP_openfile(self):
        self.GP_filename = filedialog.askopenfilename(title='open csv file', filetype=[('csv', '*.csv')])
        self.GP_entryvar.set(self.GP_filename)
        if not self.GP_filename:
            messagebox.showwarning('Warning', message='File not yet selected！')
 
    def MSG_GP_Expression(self):
        df = pd.read_csv(self.GP_filename)
        MSG_GP_x=df.iloc[:,0:2]
        MSG_GP_y=df.iloc[:,2] 
        MSG_EST_population_size=int(self.EST_population_size_Parameter.get())
        MSG_EST_generations=int(self.EST_generations_Parameter.get())
        MSG_EST_PCrossover=float(self.EST_PCrossover_Parameter.get())
        MSG_EST_PSubtreeMutation=float(self.EST_PSubtreeMutation_Parameter.get())
        MSG_EST_PHoistMutation=float(self.EST_PHoistMutation_Parameter.get())
        MSG_EST_PPointMutation=float(self.EST_PPointMutation_Parameter.get())
        feature_names = ['HR', 'N']
        MSG_est = SymbolicClassifier(population_size=MSG_EST_population_size,generations=MSG_EST_generations, stopping_criteria=0.01,
                                 function_set=('add', 'sub', 'mul', 'div', 'sin', 'tan', 'cos','sqrt',
                                       'abs','inv'),
                                   p_crossover=MSG_EST_PCrossover, p_subtree_mutation=MSG_EST_PSubtreeMutation,
                                   p_hoist_mutation=MSG_EST_PHoistMutation, p_point_mutation=MSG_EST_PPointMutation,
                                   max_samples=0.9, verbose=1,
                                   parsimony_coefficient=0.01,feature_names=feature_names)
        MSG_est.fit(MSG_GP_x,MSG_GP_y)
        MSG_GP_y_pred=MSG_est.predict(MSG_GP_x)
        MSG_GP_y_score=MSG_est.predict_proba(MSG_GP_x)[:,1]
        MSG_GP_AUC = roc_auc_score(MSG_GP_y, MSG_GP_y_score)
        MSG_GP_ACC = accuracy_score(MSG_GP_y, MSG_GP_y_pred)
        MSG_GP_PRE = precision_score(MSG_GP_y, MSG_GP_y_pred,pos_label=0)
        MSG_GP_REC = recall_score(MSG_GP_y, MSG_GP_y_pred,pos_label=0)
        MSG_GP_F1 = f1_score(MSG_GP_y, MSG_GP_y_pred,pos_label=0)      
        self.AUC_Display_GP.set(str(MSG_GP_AUC))
        self.ACC_Display_GP.set(str(MSG_GP_ACC))
        self.PRE_Display_GP.set(str(MSG_GP_PRE))
        self.REC_Display_GP.set(str(MSG_GP_REC))
        self.F1_Display_GP.set(str(MSG_GP_F1))
        self.GP_Expression_Output.set(MSG_est._program)

class ToolTip(object):
    def __init__(self, widget):
        self.widget = widget
        self.tipwindow = None
        self.id = None
        self.x = self.y = 0

    def showtip(self, text):
        self.text = text
        if self.tipwindow or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 20
        y = y + cy + self.widget.winfo_rooty() +20
        self.tipwindow = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(1)
        tw.wm_geometry("+%d+%d" % (x, y))
        label = tk.Label(tw, text=self.text, justify=tk.LEFT,
                      background="#ffffe0", relief=tk.SOLID, borderwidth=1,
                      font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hidetip(self):
        tw = self.tipwindow
        self.tipwindow = None
        if tw:
            tw.destroy()

In [30]:
if __name__ == '__main__':
    Main_menu()
    tk.mainloop()